<a href="https://colab.research.google.com/github/Hengameh1392/renewable-energy/blob/main/Copy_of_CurtailmentV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#mount google drive
import os, sys
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
dataset_path = "gdrive/MyDrive/Curtailment/AllMerged.csv"

In [ ]:
# Usual Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import sklearn
from sklearn import preprocessing

In [ ]:
df_data = pd.read_csv(dataset_path)

In [ ]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 862785 entries, 0 to 862784
Data columns (total 16 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Date                            862785 non-null  object 
 1   Hour                            862785 non-null  int64  
 2   Interval                        862785 non-null  int64  
 3   Load                            862785 non-null  int64  
 4   Solar                           862785 non-null  int64  
 5   Wind                            862785 non-null  int64  
 6   Net Load                        862785 non-null  int64  
 7   Renewables                      862785 non-null  int64  
 8   Nuclear                         862785 non-null  int64  
 9   Large Hydro                     862785 non-null  int64  
 10  Imports                         862785 non-null  int64  
 11  Generation                      862785 non-null  int64  
 12  Thermal         

In [ ]:
df_data.rename(columns = {'Wind Curtailment':'Wind_Curtailment','Solar Curtailment':'Solar_Curtailment'}, inplace = True)

In [ ]:
df_data[df_data.Wind_Curtailment!=0]

,Date,Hour,Interval,Load,Solar,Wind,Net Load,Renewables,Nuclear,Large Hydro,Imports,Generation,Thermal,Load Less (Generation+Imports),Wind_Curtailment,Solar_Curtailment
314,2014-05-02,4,2,21762,0,2770,18992,4318,2287,1115,7306,14459,6739,-3,101.0,0.0
315,2014-05-02,4,3,21726,0,2725,19001,4273,2287,1120,7319,14407,6727,0,100.0,0.0
316,2014-05-02,4,4,21885,0,2633,19252,4180,2287,1231,7528,14366,6667,-9,84.0,0.0
317,2014-05-02,4,5,22010,0,2615,19395,4165,2287,1299,7622,14387,6635,0,25.0,0.0
319,2014-05-02,4,7,21973,0,2556,19417,4108,2287,1249,7618,14357,6712,-1,74.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862690,2022-06-30,17,12,35727,12102,3381,20244,17205,2268,2469,3175,32561,10618,-10,29.0,145.0
862711,2022-06-30,19,1,36515,9496,3760,23259,14960,2266,3007,4312,32188,11954,16,97.0,0.0
862713,2022-06-30,19,2,36466,9024,3818,23624,14548,2265,3122,4478,31994,12059,-6,93.0,0.0
862715,2022-06-30,19,3,36508,8602,3905,24001,14214,2266,3184,4608,31899,12235,2,6.0,0.0


In [ ]:
df_mean=df_data.groupby(['Date','Hour']).mean()
df_mean

Interval          Load        Solar         Wind  \
Date       Hour                                                     
2014-05-01 1          6.5  23355.750000     0.000000   280.833333   
           2          6.5  22256.916667     0.000000   360.166667   
           3          6.5  21871.166667     0.000000   480.333333   
           4          6.5  21789.833333     0.000000   598.250000   
           5          6.5  22113.750000     0.000000   469.583333   
...                   ...           ...          ...          ...   
2022-06-30 20         6.5  35761.333333  1736.416667  3784.583333   
           21         6.5  34867.666667    15.666667  3772.083333   
           22         6.5  33819.500000     0.000000  3992.166667   
           23         6.5  31523.000000     0.000000  4164.333333   
           24         6.5  29020.916667     0.000000  4091.250000   

                     Net Load   Renewables      Nuclear  Large Hydro  \
Date       Hour                                                        
2014-05-01 1     23074.666667  1848.416667  2287.250000  1399.083333   
           2     21896.750000  1919.833333  2287.500000  1115.166667   
           3     21390.750000  2010.666667  2286.083333  1269.750000   
           4     21191.500000  2135.166667  2286.500000  1382.083333   
           5     21644.250000  2007.750000  2287.416667  1130.666667   
...                       ...          ...          ...          ...   
2022-06-30 20    30240.666667  7240.083333  2265.083333  4294.333333   
           21    31079.916667  5553.416667  2266.083333  4271.750000   
           22    29827.333333  5766.333333  2266.916667  3969.333333   
           23    27358.750000  5862.333333  2268.500000  3312.250000   
           24    24929.500000  5784.833333  2270.416667  2911.000000   

                     Imports    Generation       Thermal  \
Date       Hour                                            
2014-05-01 1     7413.166667  15940.333333  10405.750000   
           2     7399.666667  14857.916667   9535.833333   
           3     7468.333333  14402.583333   8836.166667   
           4     7436.250000  14353.250000   8549.750000   
           5     7536.333333  14576.833333   9151.333333   
...                      ...           ...           ...   
2022-06-30 20    4850.083333  30904.833333  17105.500000   
           21    4643.166667  30219.416667  18128.166667   
           22    5272.416667  28538.250000  16536.000000   
           23    6193.666667  25311.250000  13868.250000   
           24    7081.250000  21926.333333  10960.166667   

                 Load Less (Generation+Imports)  Wind_Curtailment  \
Date       Hour                                                     
2014-05-01 1                           2.083333              0.00   
           2                          -0.666667              0.00   
           3                           0.250000              0.00   
           4                           0.166667              0.00   
           5                           0.750000              0.00   
...                                         ...               ...   
2022-06-30 20                          6.583333              0.00   
           21                          5.250000              2.25   
           22                          8.666667              0.00   
           23                         17.916667              0.00   
           24                         13.250000              0.00   

                 Solar_Curtailment  
Date       Hour                     
2014-05-01 1                   0.0  
           2                   0.0  
           3                   0.0  
           4                   0.0  
           5                   0.0  
...                            ...  
2022-06-30 20                  0.0  
           21                  0.0  
           22                  0.0  
           23                  0.0  
           24                  0.0  

[71569 rows x 14 colu

In [ ]:
df_mean = df_mean.reset_index()
df_mean

,Date,Hour,Interval,Load,Solar,Wind,Net Load,Renewables,Nuclear,Large Hydro,Imports,Generation,Thermal,Load Less (Generation+Imports),Wind_Curtailment,Solar_Curtailment
0,2014-05-01,1,6.5,23355.750000,0.000000,280.833333,23074.666667,1848.416667,2287.250000,1399.083333,7413.166667,15940.333333,10405.750000,2.083333,0.00,0.0
1,2014-05-01,2,6.5,22256.916667,0.000000,360.166667,21896.750000,1919.833333,2287.500000,1115.166667,7399.666667,14857.916667,9535.833333,-0.666667,0.00,0.0
2,2014-05-01,3,6.5,21871.166667,0.000000,480.333333,21390.750000,2010.666667,2286.083333,1269.750000,7468.333333,14402.583333,8836.166667,0.250000,0.00,0.0
3,2014-05-01,4,6.5,21789.833333,0.000000,598.250000,21191.500000,2135.166667,2286.500000,1382.083333,7436.250000,14353.250000,8549.750000,0.166667,0.00,0.0
4,2014-05-01,5,6.5,22113.750000,0.000000,469.583333,21644.250000,2007.750000,2287.416667,1130.666667,7536.333333,14576.833333,9151.333333,0.750000,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71564,2022-06-30,20,6.5,35761.333333,1736.416667,3784.583333,30240.666667,7240.083333,2265.083333,4294.333333,4850.083333,30904.833333,17105.500000,6.583333,0.00,0.0
71565,2022-06-30,21,6.5,34867.666667,15.666667,3772.083333,31079.916667,5553.416667,2266.083333,4271.750000,4643.166667,30219.416667,18128.166667,5.250000,2.25,0.0
71566,2022-06-30,22,6.5,33819.500000,0.000000,3992.166667,29827.333333,5766.333333,2266.916667,3969.333333,5272.416667,28538.250000,16536.000000,8.666667,0.00,0.0
71567,2022-06-30,23,6.5,31523.000000,0.000000,4164.333333,27358.750000,5862.333333,2268.500000,3312.250000,6193.666667,25311.250000,13868.250000,17.916667,0.00,0.0


In [ ]:
delay=1
test_precent=0.20

In [ ]:
X=df_mean.drop(['Date','Hour'],axis=1)[:-delay].reset_index(drop=True)
y_wind=df_mean['Wind_Curtailment'][delay:].reset_index(drop=True)
y_solar=df_mean['Solar_Curtailment'][delay:].reset_index(drop=True)

In [ ]:
y_wind

0        0.00
1        0.00
2        0.00
3        0.00
4        0.00
         ... 
71563    0.00
71564    2.25
71565    0.00
71566    0.00
71567    0.00
Name: Wind_Curtailment, Length: 71568, dtype: float64

In [ ]:
train_ind=int(len(X)*(1-test_precent))
X_train=X[:train_ind]
X_test=X[train_ind:]
y_wind_train=y_wind[:train_ind]
y_wind_test=y_wind[train_ind:]
y_solar_train=y_solar[:train_ind]
y_solar_test=y_solar[train_ind:]


In [ ]:
from sklearn import svm
from sklearn.neural_network import MLPRegressor
from sklearn import tree
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_absolute_error,mean_absolute_percentage_error


from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier, XGBRFClassifier
from xgboost import plot_tree, plot_importance

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test=min_max_scaler.transform(X_test)

In [ ]:
def model_assess(model,X_train,y_train,y_test,title = "Default"):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    #print(confusion_matrix(y_test, preds))
    print('MAE', title, ':', mean_absolute_error(y_test, preds), '\n')

In [ ]:
# Support Vector Machine
model= svm.SVR()
model_assess(model,X_train,y_wind_train,y_wind_test,"Support Vector Machine")


MAE Support Vector Machine : 10.814959432659272 



In [ ]:
#MLPRegressor
model = MLPRegressor(random_state=1, max_iter=500)
model_assess(model,X_train,y_wind_train,y_wind_test,"MLPRegressor")

MAE MLPRegressor : 8.748575156995486 



In [ ]:
#Regression Tree
model = tree.DecisionTreeRegressor()
model_assess(model,X_train,y_wind_train,y_wind_test,"Regression Tree")

MAE Regression Tree : 13.496324715504569 



In [ ]:
#Crash on RAM!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#Gaussian Process Regression
#kernel = DotProduct() + WhiteKernel()
#model = GaussianProcessRegressor(kernel=kernel,random_state=0)

#model_assess(model,X_train,y_wind_train,y_wind_test,"Gaussian Process Regression")

In [ ]:
#Regression based on k-nearest neighbors
model = KNeighborsRegressor(n_neighbors=5)
model_assess(model,X_train,y_wind_train,y_wind_test,"Regression based on k-nearest neighbors")

MAE Regression based on k-nearest neighbors : 13.189226273296546 



In [ ]:
#SGDRegressor
model=SGDRegressor(max_iter=1000, tol=1e-3)
model_assess(model,X_train,y_wind_train,y_wind_test,"SGDRegressor")

MAE SGDRegressor : 10.895454296446633 



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1507: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


In [ ]:

# Support Vector Machine
model= svm.SVR()
model_assess(model,X_train,y_solar_train,y_solar_test,"Support Vector Machine")

#MLPRegressor
model = MLPRegressor(random_state=1, max_iter=500)
model_assess(model,X_train,y_solar_train,y_solar_test,"MLPRegressor")

#Regression Tree
model = tree.DecisionTreeRegressor()
model_assess(model,X_train,y_solar_train,y_solar_test,"Regression Tree")

#Regression based on k-nearest neighbors
model = KNeighborsRegressor(n_neighbors=5)
model_assess(model,X_train,y_solar_train,y_solar_test,"Regression based on k-nearest neighbors")

#SGDRegressor
model=SGDRegressor(max_iter=1000, tol=1e-3)
model_assess(model,X_train,y_solar_train,y_solar_test,"SGDRegressor")

MAE Support Vector Machine : 187.28874780656392 

MAE MLPRegressor : 93.91985769667737 

MAE Regression Tree : 116.42733908057109 

MAE Regression based on k-nearest neighbors : 130.77569194936856 

MAE SGDRegressor : 97.2192669820135 

